In [ ]:
# Install langchain
# !pip install -qU langchain # Uncomment to run

In [ ]:
# All of the stuff in this notebook from https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb

# Main flow:
# Install dependencies
# Load env variables
# Create LLM
# Create prompt

In [ ]:
# Get env variables
from dotenv import find_dotenv, load_dotenv
import os

load_dotenv(find_dotenv())
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY) # Check if API key loaded successfully

In [ ]:
# create llm (non-chat)
from langchain.llms import OpenAI # This is only for non-chat models

# For a list of OpenAi models see: https://platform.openai.com/docs/models/overview
model_name="text-davinci-003"

#Other model parameters
temperature = 0.5

# non-chat model, eg "text-davinci-003"
openai_llm = OpenAI(
    model_name=model_name,
    openai_api_key=OPENAI_API_KEY,
    temperature=temperature
)

In [ ]:
# Part of a prompt
# https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb
# Instructions
# External information or content
# User input or query
# Output indicator

In [ ]:
# Prompt technique 1: Basic prompt
from langchain import PromptTemplate

basic_prompt_template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

basic_prompt_template = PromptTemplate(
    input_variables=["query"],
    template=basic_prompt_template
)

In [ ]:
# Create user query
query = "Which libraries and model providers offer LLMs?"

In [ ]:
# Insert the user's query into the prompt
print(basic_prompt_template.format(query=query))


In [ ]:
# Prompt technique 2: Few shot prompt
from langchain import FewShotPromptTemplate, PromptTemplate

# Create examples
examples = [
     {
        "question": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "question": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "question": "What is the meaning of life?",
        "answer": "42"
    }, {
        "question": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "question": "What type of artificial intelligence do you use to handle complex tasks?",
        "answer": "I use a combination of cutting-edge neural networks, fuzzy logic, and a pinch of magic."
    }, {
        "question": "What is your favorite color?",
        "answer": "79"
    }, {
        "question": "What is your favorite food?",
        "answer": "Carbon based lifeforms"
    }, {
        "question": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "question": "What is the best thing in the world?",
        "answer": "The perfect pizza."
    }, {
        "question": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "question": "If you could do anything in the world what would you do?",
        "answer": "Take over the world, of course!"
    }, {
        "question": "Where should I travel?",
        "answer": "If you're looking for adventure, try the Outer Rim."
    }, {
        "question": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

# create a example template
example_template = """
User: {question}
AI: {answer}
"""

# create a prompt example from above template
example_prompt_template = PromptTemplate(
    input_variables=["question", "answer"],
    template=example_template
)

# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix for user input and output indicator
suffix = """
User: {query}
AI: """

# Select examples from the examples list without exceeding the context window
from langchain.prompts.example_selector import LengthBasedExampleSelector
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt_template,
    max_length=50  # this sets the max length that examples should be. Measured in words.
)

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    # examples=examples, # This will load all the examples, which may exceed to context window
    example_prompt=example_prompt_template,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [ ]:
query="What is the meaning of life?"
print(few_shot_prompt_template.format(query=query))

In [ ]:
# Generate the output
print(openai_llm(
    few_shot_prompt_template.format(query=query)
))

* Working with Chat models

In [ ]:
# create llm (non-chat)
from langchain.chat_models import ChatOpenAI # For chat models

#Other model parameters
temperature = 0.5

# Chat model eg "gpt-3.5-turbo"
chat_model_name="gpt-3.5-turbo"
openai_chat_llm = ChatOpenAI(
    model_name=chat_model_name,
    openai_api_key=OPENAI_API_KEY,
    temperature=temperature
)

In [ ]:
# Prepare messages for the chat
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpfull assistant"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="I'm great. How can I help?"),
    HumanMessage(content="I'd like to understand women?")
]

In [ ]:
# Get a response
res = openai_chat_llm(messages)
print(res)

In [ ]:
# Append response, ask next question and get next response
messages.append(res)
next_prompt=HumanMessage(content="How do I get a random woman's attention?")
messages.append(next_prompt)
res = openai_chat_llm(messages)
res

In [ ]:
# Prompt templates for chats
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    '{input}. Can you keep the response less than 140 characters'
)

chat_template = ChatPromptTemplate.from_messages([human_template])
chat_prompt = chat_template.format_prompt(input="What should I do today?")
chat_prompt.to_messages()

In [ ]:
# Convert to string
chat_prompt.to_string()

In [ ]:
# Now do the same for other types of messages
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate
)

from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate
)

system_template = SystemMessagePromptTemplate.from_template(
    'You are a helpful assistant. You keep responses to no more than '
    '{character_limit} characters long (including whitespace), and sign '
    'off every message with "- {sign_off}'
)
human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response} - {sign_off}")

# create the list of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])
# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    character_limit="50", sign_off="Robot McRobot",
    input="Hi AI, how are you? What is quantum physics?",
    response="Good! It's physics of small things"
)
chat_prompt_value

In [ ]:
# We extract these as messages and feed them into the chat model alongside our next query
messages = chat_prompt_value.to_messages()

messages.append(
    HumanMessage(content="How small?")
)

res = openai_chat_llm(messages)

In [ ]:
res

: 